<a href="https://www.kaggle.com/code/jimschacko/bidirectional-lstm-rnn-fake-news-model?scriptVersionId=135621511" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#fake news prediction using Bidirectional LSTM RNN
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/kaggle/input/fake-news/train.csv')

In [4]:
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [5]:
df.shape

(20800, 5)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [7]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [8]:
df = df.dropna()

In [9]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [10]:
x = df.drop('label', axis =1)
y = df['label']

In [11]:
#balce dataset or not
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [12]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [13]:
voc_size = 5000

In [14]:
messages = x.copy()

In [15]:
import nltk
from nltk.corpus import stopwords
import re

In [16]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
messages.reset_index(inplace=True)

In [18]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word)for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [19]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [20]:
onehot = [one_hot(words,voc_size)for words in corpus]

In [21]:
print(onehot)

[[2403, 1216, 1037, 1574, 1504, 3646, 2298, 2866, 267, 1868], [909, 4321, 3013, 3689, 2638, 4283, 4861], [2314, 1912, 761, 4619], [717, 2147, 2342, 822, 4864, 3589], [42, 2638, 3795, 1634, 3325, 1048, 2638, 3554, 365, 4419], [4391, 201, 3022, 4787, 1450, 3613, 4737, 199, 851, 1590, 1729, 4865, 3015, 4787, 4861], [1522, 4356, 3754, 4414, 1873, 4036, 1699, 1519, 409, 3688, 3117], [2624, 3803, 3743, 4449, 1404, 1078, 3613, 3663, 409, 3688, 3117], [3249, 1191, 1725, 3299, 1778, 1485, 4413, 1921, 3613, 2032], [1640, 3041, 4480, 4663, 2994, 316, 3902, 1353], [499, 2338, 350, 2516, 1676, 1198, 4326, 4509, 3456, 783, 4535], [822, 423, 1504, 1485, 3613, 1404], [177, 3122, 2670, 2616, 1921, 4684, 4022, 2686, 2687], [4715, 1470, 4592, 21, 1078, 71, 4008, 409, 3688, 3117], [1443, 3969, 2907, 1853, 2280, 409, 3688, 3117], [1467, 1344, 4433, 4881, 309, 4213, 2475, 3696, 2068, 3874], [4098, 688, 4321], [2349, 4492, 24, 4915, 3613, 1197, 3698, 4861], [4043, 527, 3013, 4891, 2131, 3304, 3202, 2316, 104

In [22]:
sent_length = 15

In [23]:
embedded_doc = pad_sequences(onehot,padding='pre', maxlen=sent_length)

In [24]:
print(embedded_doc)

[[   0    0    0 ... 2866  267 1868]
 [   0    0    0 ... 2638 4283 4861]
 [   0    0    0 ... 1912  761 4619]
 ...
 [   0    0    0 ...  409 3688 3117]
 [   0    0    0 ... 3736 3261 4528]
 [   0    0    0 ...  941  596 2979]]


In [25]:
embedding_vector_features = 40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [26]:
import numpy as np
x_final = np.array(embedded_doc)
y_final = np.array(y)

In [27]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_final,y_final,test_size=0.33,random_state=0)

In [28]:
model.fit(x_train,y_train,validation_data=(x_test,y_test), epochs = 10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 11s 37ms/step - loss: 0.2896 - accuracy: 0.8615 - val_loss: 0.2034 - val_accuracy: 0.9201
Epoch 2/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1426 - accuracy: 0.9472 - val_loss: 0.2163 - val_accuracy: 0.9167
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1000 - accuracy: 0.9635 - val_loss: 0.2214 - val_accuracy: 0.9128
Epoch 4/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0699 - accuracy: 0.9764 - val_loss: 0.2581 - val_accuracy: 0.9137
Epoch 5/10
192/192 [==============================] - 7s 34ms/step - loss: 0.0459 - accuracy: 0.9857 - val_loss: 0.2769 - val_accuracy: 0.9062
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0292 - accuracy: 0.9922 - val_loss: 0.3964 - val_accuracy: 0.9074
Epoch 7/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0196 - accuracy: 0.9949 - val_loss: 0.4448 - val_accuracy: 0.907

In [29]:
y_pred = model.predict(x_test)

189/189 [==============================] - 2s 5ms/step


In [30]:
y_pred = np.where(y_pred>=0.5,1,0)
print(y_pred)

[[1]
 [1]
 [1]
 ...
 [0]
 [0]
 [0]]


In [31]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_pred,y_test)

array([[3094,  243],
       [ 295, 2403]])

In [32]:
print(accuracy_score(y_test,y_pred))


0.9108533554266777
